# import

In [2]:
# Setup - Run only once per Kernel App
%conda install https://anaconda.org/conda-forge/openjdk/11.0.1/download/linux-64/openjdk-11.0.1-hacce0ff_1021.tar.bz2

# install PySpark
!pip install sagemaker_pyspark
%pip install pyspark==3.4.0

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")


                                                                                


Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.
  Using cached sagemaker_pyspark-1.4.5-py3-none-any.whl
  Using cached pyspark-3.3.0-py2.py3-none-any.whl
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
  Using cached pyspark-3.4.0-py2.py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.3.0
    Uninstalling pyspark-3.3.0:
      Successfully uninstalled pyspark-3.3.0
ERROR: pip's dependency resolver does not currentl

In [3]:
import sagemaker
sess = sagemaker.Session()
bucket = sess.default_bucket() 
print(f"the default SageMaker region specific bucket for this account is {bucket}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
the default SageMaker region specific bucket for this account is sagemaker-us-east-1-165729782536


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("sagemaker-spark") \
    .master("local[*]") \
    .config("spark.driver.memory", "8G") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3,org.apache.hadoop:hadoop-aws:3.2.2") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.ContainerCredentialsProvider") \
    .getOrCreate()

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9ff57a08-1ee8-4cdb-add0-e4f299f94e75;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in c

3.4.0


In [9]:
import pyspark.sql.functions as f

## read in data

In [6]:
# Define the path to the CSV file
s3_path = f"s3a://{bucket}/project/covid_data/epidemiology.csv"

# Read the CSV file
epi_data = spark.read.csv(s3_path, header=True, inferSchema=True)

# Show the DataFrame to verify the content
epi_data.show()


24/04/27 16:33:51 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+----------+------------+-------------+------------+-------------+----------+--------------------+-------------------+--------------------+-----------------+
|      date|location_key|new_confirmed|new_deceased|new_recovered|new_tested|cumulative_confirmed|cumulative_deceased|cumulative_recovered|cumulative_tested|
+----------+------------+-------------+------------+-------------+----------+--------------------+-------------------+--------------------+-----------------+
|2020-01-01|          AD|            0|           0|         null|      null|                   0|                  0|                null|             null|
|2020-01-02|          AD|            0|           0|         null|      null|                   0|                  0|                null|             null|
|2020-01-03|          AD|            0|           0|         null|      null|                   0|                  0|                null|             null|
|2020-01-04|          AD|            0|           0|

In [7]:
epi_data.printSchema()

root
 |-- date: date (nullable = true)
 |-- location_key: string (nullable = true)
 |-- new_confirmed: integer (nullable = true)
 |-- new_deceased: integer (nullable = true)
 |-- new_recovered: integer (nullable = true)
 |-- new_tested: integer (nullable = true)
 |-- cumulative_confirmed: integer (nullable = true)
 |-- cumulative_deceased: integer (nullable = true)
 |-- cumulative_recovered: integer (nullable = true)
 |-- cumulative_tested: long (nullable = true)



In [8]:
file = "google-search-trends.csv"

s3_path = f"s3a://{bucket}/project/covid_data/{file}"

# Read the CSV file
search_data = spark.read.csv(s3_path, header=True, inferSchema=True)

# Show the DataFrame to verify the content
search_data.show()
search_data.printSchema()

24/04/27 16:38:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+------------+-------------------------------+----------------------------+------------------+-------------------------------+------------------------------+----------------------------+---------------------+------------------------+-------------------------------------+---------------------+-----------------------+------------------------+---------------------+--------------------------+-------------------------+--------------------+-----------------------------+------------------------+-------------------------------+---------------------+---------------------+---------------------+---------------------+-------------------+------------------------+-----------------------+---------------------+-------------------------------+----------------------+--------------------+-------------------------+--------------------+----------------------+------------------------------------------------------+------------------------------------+--------------------------------+--------------

In [12]:
print(search_data.columns)

['date', 'location_key', 'search_trends_abdominal_obesity', 'search_trends_abdominal_pain', 'search_trends_acne', 'search_trends_actinic_keratosis', 'search_trends_acute_bronchitis', 'search_trends_adrenal_crisis', 'search_trends_ageusia', 'search_trends_alcoholism', 'search_trends_allergic_conjunctivitis', 'search_trends_allergy', 'search_trends_amblyopia', 'search_trends_amenorrhea', 'search_trends_amnesia', 'search_trends_anal_fissure', 'search_trends_anaphylaxis', 'search_trends_anemia', 'search_trends_angina_pectoris', 'search_trends_angioedema', 'search_trends_angular_cheilitis', 'search_trends_anosmia', 'search_trends_anxiety', 'search_trends_aphasia', 'search_trends_aphonia', 'search_trends_apnea', 'search_trends_arthralgia', 'search_trends_arthritis', 'search_trends_ascites', 'search_trends_asperger_syndrome', 'search_trends_asphyxia', 'search_trends_asthma', 'search_trends_astigmatism', 'search_trends_ataxia', 'search_trends_atheroma', 'search_trends_attention_deficit_hyperac

In [19]:
#Global_vaccination_search_insights.csv

file = "Global_vaccination_search_insights.csv"

s3_path = f"s3a://{bucket}/project/covid_data/{file}"

# Read the CSV fil
vaccine_search = spark.read.csv(s3_path, header=True, inferSchema=True)

# Show the DataFrame to verify the content
# vaccine_data.show()
vaccine_search.printSchema()

root
 |-- date: date (nullable = true)
 |-- country_region: string (nullable = true)
 |-- country_region_code: string (nullable = true)
 |-- sub_region_1: string (nullable = true)
 |-- sub_region_1_code: string (nullable = true)
 |-- sub_region_2: string (nullable = true)
 |-- sub_region_2_code: integer (nullable = true)
 |-- sub_region_3: string (nullable = true)
 |-- sub_region_3_code: string (nullable = true)
 |-- place_id: string (nullable = true)
 |-- sni_covid19_vaccination: double (nullable = true)
 |-- sni_vaccination_intent: double (nullable = true)
 |-- sni_safety_side_effects: double (nullable = true)



In [20]:
# Define the path to the CSV file
s3_path = f"s3a://{bucket}/project/covid_data/vaccinations.csv"

# Read the CSV file
vac_data = spark.read.csv(s3_path, header=True, inferSchema=True)

# Show the DataFrame to verify the content
vac_data.printSchema()


root
 |-- date: date (nullable = true)
 |-- location_key: string (nullable = true)
 |-- new_persons_vaccinated: integer (nullable = true)
 |-- cumulative_persons_vaccinated: integer (nullable = true)
 |-- new_persons_fully_vaccinated: integer (nullable = true)
 |-- cumulative_persons_fully_vaccinated: integer (nullable = true)
 |-- new_vaccine_doses_administered: integer (nullable = true)
 |-- cumulative_vaccine_doses_administered: long (nullable = true)
 |-- new_persons_vaccinated_pfizer: integer (nullable = true)
 |-- cumulative_persons_vaccinated_pfizer: integer (nullable = true)
 |-- new_persons_fully_vaccinated_pfizer: integer (nullable = true)
 |-- cumulative_persons_fully_vaccinated_pfizer: integer (nullable = true)
 |-- new_vaccine_doses_administered_pfizer: integer (nullable = true)
 |-- cumulative_vaccine_doses_administered_pfizer: integer (nullable = true)
 |-- new_persons_vaccinated_moderna: integer (nullable = true)
 |-- cumulative_persons_vaccinated_moderna: integer (null

### data preprocess

In [28]:
def add_time_columns(df, date_col='date'):

    return df.withColumn("year", f.year(date_col)) \
             .withColumn("month", f.month(date_col)) \
             .withColumn("week", f.weekofyear(date_col))

In [29]:
def group_by_weekly(df, cols):

    # Prepare aggregation expressions
    aggregations = {col: "sum" for col in cols}
    

    return df.groupBy("year", "week").agg(
        *(f.sum(c).alias(f"sum_{c}") for c in cols)
    ).orderBy("year", "week")

In [30]:
def group_by_monthly(df, cols):

    aggregations = {col: "sum" for col in cols}
    
    # Group by year and month, then aggregate based on the provided columns
    return df.groupBy("year", "month").agg(
        *(f.sum(c).alias(f"sum_{c}") for c in cols)
    ).orderBy("year", "month")


In [31]:
# Extract the month from the date and group by it
epi_data = add_time_columns(epi_data)

epi_agg_cols =  ["new_confirmed", "new_deceased"]
monthly_epi_global = group_by_monthly(epi_data, epi_agg_cols)
weekly_epi_global = group_by_weekly(epi_data, epi_agg_cols)




In [32]:
# Extract the month from the date and group by it
vac_data = add_time_columns(vac_data)

vac_agg_cols =  ["new_persons_vaccinated", "new_persons_fully_vaccinated", "new_vaccine_doses_administered"]
monthly_vac_global = group_by_monthly(vac_data, vac_agg_cols)
weekly_vac_global = group_by_weekly(vac_data, vac_agg_cols)




In [34]:
weekly_vac_global.show()

+----+----+--------------------------+--------------------------------+----------------------------------+
|year|week|sum_new_persons_vaccinated|sum_new_persons_fully_vaccinated|sum_new_vaccine_doses_administered|
+----+----+--------------------------+--------------------------------+----------------------------------+
|2020|   4|                      null|                               0|                              null|
|2020|   5|                      null|                               0|                              null|
|2020|   6|                      null|                               0|                              null|
|2020|   7|                      null|                               0|                              null|
|2020|   8|                      null|                               0|                              null|
|2020|   9|                      null|                               0|                              null|
|2020|  10|                      null